In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from bokeh.plotting import figure, show
from bokeh.io import output_file

In [3]:
#la funcion dow, descarga todos los archivos contenidos en los links de la variable url

def dow(url,cat):
    
    for i in range(len(url)):
        html = requests.get(url[i]).content
        x_list = pd.read_html(html,header=0)
        x = x_list[1]
        cat.append(x.columns[1])
        globals()[cat[i]] = x
        globals()[cat[i]] = pd.Series(globals()[x.columns[1]][x.columns[1]].values,
                                     index=globals()[x.columns[1]]['Country'])

In [4]:
url = ['https://www.indexmundi.com/map/?t=0&v=2226&r=xx&l=en',
       'https://www.indexmundi.com/map/?t=0&v=26&r=xx&l=en',
       'https://www.indexmundi.com/map/?t=0&v=25&r=xx&l=en']
cat = []
t=[]
file='BvD.csv'

In [5]:
dow(url,cat)

In [6]:
#Este ciclo crea una variable que contiene los comandos para llamar a cada columna de datos
#obtenidos con dow

l = list(range(len(url)))
l = [str(c) for c in l]
for i in range(len(url)):
    j=str(i)
    l[i]='cat['+j+']:globals()[cat['+j+']]'

#Este ciclo crea una linea de codigo para unir todas las tablas en una sola
#organizada por pais
k='t = pd.DataFrame({'
for i in range(len(url)):
    k=k+ l[i] +', '
    
k=k+'})'

#exec ejecuta la linea de codigo generada en el ciclo anterior    
exec(k)

In [7]:
#Guarda los datos en un archivo
t.to_csv(file)

# Plot

In [8]:

fig = figure(plot_height=400,
             plot_width=600,
             x_axis_label=t.columns[1],
             y_axis_label=t.columns[2],
             title='Deaths vs Births',
             toolbar_location='below',

            )

tooltips = [
            ('Country', '@index'),
#            ('Deaths', '@'+t.columns[0]),
#            ('Births', '@'+t.columns[1]),
#            ('Physicians', '@'+t.columns[2]),
           ]

fig.add_tools(HoverTool(tooltips=tooltips))

fig.square(x=t.columns[1], y=t.columns[2], 
           source=t, color='royalblue', 
           selection_color='deepskyblue', nonselection_color='lightgray', nonselection_alpha=0.3)


show(fig)